In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from collections import defaultdict
from tqdm import tqdm 
from heapq import nlargest

tqdm.pandas()

In [2]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [3]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [4]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 14647/14647 [00:00<00:00, 38929.74it/s]


In [5]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [6]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]

In [7]:
career_paths

candidate_id
84556      [208.0, 348.0, 208.0, 344.0, 344.0, 344.0, 344...
84612      [344.0, 335.0, 332.0, 344.0, 335.0, 222.0, 344.0]
84731      [344.0, 208.0, 208.0, 345.0, 229.0, 345.0, 323.0]
85437      [344.0, 323.0, 344.0, 185.0, 177.0, 185.0, 185...
85888                                  [229.0, 229.0, 208.0]
                                 ...                        
2712644    [142.0, 193.0, 344.0, 345.0, 345.0, 323.0, 323...
2713636    [345.0, 344.0, 344.0, 345.0, 345.0, 345.0, 344...
2713846                  [208.0, 208.0, 177.0, 177.0, 187.0]
2714059      [208.0, 207.0, 150.0, 24.0, 229.0, 353.0, 24.0]
2714264    [199.0, 208.0, 208.0, 208.0, 208.0, 208.0, 208...
Name: isco_code4, Length: 6420, dtype: object

In [8]:
switches = defaultdict(lambda: defaultdict(int))

for c, jobs in career_paths.iteritems():
    for i in range(len(jobs)):
        if i < len(jobs) - 1 and jobs[i] != jobs[i + 1]:
            switches[jobs[i]][jobs[i  + 1]] += 1

In [13]:
most_common = pd.Series({k: set(nlargest(5, v, key=v.get)) for k, v in switches.items()})

In [14]:
most_common.head()

208.0    {229.0, 335.0, 177.0, 345.0, 348.0}
348.0    {229.0, 335.0, 208.0, 207.0, 345.0}
344.0    {323.0, 332.0, 335.0, 343.0, 345.0}
345.0    {195.0, 332.0, 335.0, 177.0, 344.0}
185.0    {227.0, 174.0, 177.0, 344.0, 345.0}
dtype: object

In [15]:
most_common_switch = career_paths.progress_apply(lambda x: x[-1] in most_common.loc[x[-2]]).mean()

100%|██████████| 6420/6420 [00:00<00:00, 35282.69it/s]


In [16]:
print(f"Most common switch accuracy: {most_common_switch * 100:>.2f}%")

Most common switch accuracy: 55.08%
